<a href="https://colab.research.google.com/github/hahajjjun/Disastogram_Project/blob/main/NLP_API_Request_and_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib3
import json
import pandas as pd

In [23]:
data = pd.read_excel("/content/drive/MyDrive/NLP_Project/covid_autolabeled.xlsx")
# 언어 분석 기술 문어/구어 중 한가지만 선택해 사용
# 언어 분석 기술(문어)
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 
# 언어 분석 기술(구어)
# openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"
 
accessKey = "49a01d9e-396d-43e8-b1f6-7803fef625d2"
analysisCode = "ner"
text = ""

In [24]:
query = [',','(',')','<','>','~','.','-','/']
total = []
for msg in data['텍스트']:
    count = 0
    keyword = []
    text = msg
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }
    
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
    res = str(response.data,"utf-8")
    json_res = json.loads(res)
    
    dic = {}
    for item in json_res['return_object']['sentence'][0]['morp']:
        if item['lemma'] in dic.keys():
            dic[item['lemma']]+=1
        else:
            dic[item['lemma']]=1
    temp = sorted(dic.items(), key=lambda item: item[1], reverse=True)
    for tup in temp:
        if tup[0] not in query and count<=2:
            keyword.append(tup[0])
            count = count+1
    total.append(keyword)
data['embedding']=total
data

,Unnamed: 0,장소,텍스트,클래스,embedding,세부 분류
0,7,단양군청,7.3.(토) 08:30 동서울->단양/7.4.(일) 17:30 단양->동서울 버스...,2,"[:, 30, 동서울]",NaN
1,12,화성시청,6. 27.(일)~6. 30.(수) 세시봉(송산면 사강로 208) 방문자는 주소지...,2,"[6, 27, 일]",NaN
2,23,전남도청,6.16.~28. 서울 마포구 홍대 주변 음식점(라밤바/젠바/도깨비클럽/FF클럽/...,2,"[6.16, 28]",NaN
3,30,광주광역시,"6.26(토), 6.28(월), 6.29(화) 오로지 백반만(남구 금화로409, ...",2,"[6.26, 토, 6.28]",NaN
4,32,의정부시청,6.17.~6.30.제일시장 지하 음식백화점(태평로73번길 20) 방문자는 선별진료...,2,"[6.17, 6.30, 제일]",NaN
...,...,...,...,...,...,...
202,181,전남도청,"8.26. 24시기준 확진16명(여수4,순천5,담양1,장흥3,해남1,장성2)▲18...",5,"[시, 1, 8.26]",NaN
203,190,전남도청,"8.13.24시기준 확진자28명(목포5,여수2,순천4,나주5,광양3,보성2,해남1...",5,"[1, 5, 2]",NaN
204,191,전남도청,"8.4(수)24시기준 확진자20명(목포3,여수3,순천5,나주3,광양1,고흥3,강진1...",5,"[3, 1, ▲]",NaN
205,195,전남도청,"8.6(금) 24시 기준 확진자 20명(목포1,여수5,순천2,나주1,담양1,고흥1...",5,"[1, 4, 8.6]",NaN


In [25]:
data.to_excel('NLPmodel_embedding.xlsx')